# Form the NB Regression Random Intercept Model: Simulated set

Using Block update of beta and MM-update. We will use the file "fit_old.jl"

Without having to compute the gradient and hessian of variance components vector, we finish in 67 iterations and 9 seconds.

Next we may try joint estimation using Newton's + IPOPT. 

In [68]:
using Revise
using DataFrames, Random, GLM, GLMCopula
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!
using SpecialFunctions

Random.seed!(1234)

# sample size
N = 10000
# observations per subject
n = 5

variance_component_1 = 0.9
variance_component_2 = 0.1

r = 1
p = 0.7
μ = r * (1-p) * inv(p)

#var = r * (1-p) * inv(p^2)

# true beta
β_true = log(μ)

dist = NegativeBinomial

Γ = variance_component_1 * ones(n, n) + variance_component_2 * Matrix(I, n, n)
vecd = [dist(r, p) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

10000-element Vector{Vector{Float64}}:
 [0.0, 1.0, 0.0, 0.0, 1.0]
 [2.0, 0.0, 0.0, 0.0, 1.0]
 [1.0, 0.0, 0.0, 4.0, 1.0]
 [0.0, 0.0, 0.0, 0.0, 1.0]
 [3.0, 3.0, 3.0, 1.0, 1.0]
 [0.0, 0.0, 0.0, 0.0, 0.0]
 [3.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 1.0, 0.0, 0.0, 2.0]
 [0.0, 1.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0, 0.0]
 ⋮
 [0.0, 0.0, 1.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 1.0, 0.0, 0.0, 1.0]
 [0.0, 0.0, 1.0, 2.0, 0.0]
 [1.0, 0.0, 0.0, 0.0, 3.0]
 [0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 1.0, 2.0, 1.0]
 [2.0, 2.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 1.0, 0.0]
 [2.0, 1.0, 2.0, 0.0, 0.0]
 [0.0, 0.0, 1.0, 0.0, 0.0]

In [69]:
d = NegativeBinomial()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaVCObs{T, D, Link}}(undef, N)
for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)]
    gcs[i] = GLMCopulaVCObs(y, X, V, d, link)
end
gcm = GLMCopulaVCModel(gcs);

In [70]:
initialize_model!(gcm)
@show gcm.β
@show gcm.Σ
@show gcm.data[1].d.r

initializing β using Newton's Algorithm under Independence Assumption
1 0.0 -57421.69453199051 39999
2 -57421.69453199051 -57421.69453199051 39999
initializing variance components using MM-Algorithm
gcm.β = [-0.3620034006976286]
gcm.Σ = [0.18367019483213523, 4.683717988259242e-5]
(gcm.data[1]).d.r = 1.0


1.0

Initialize β and σ2, here I just copy the solution for β and σ2 from MixedModels.jl over

In [71]:
GLMCopula.loglikelihood!(gcm, true, true)

-56845.87658829459

### Fit

In [72]:
# does not update r
@time fit2!(gcm, IpoptSolver(print_level = 5, max_iter = 500, mehrotra_algorithm="yes", warm_start_init_point="yes", hessian_approximation = "exact"))

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        4



LoadError: DomainError with -4.009759181267438:
log will only return a complex result if called with a complex argument. Try log(Complex(x)).

In [55]:
# updates r using newton
@time fit!(gcm, IpoptSolver(print_level = 5, max_iter = 500, mehrotra_algorithm="yes", warm_start_init_point="yes", hessian_approximation = "exact"))

new_d = NegativeBinomial{Float64}(r=2.1996724830149925, p=0.5)
gcm.θ = [0.6698378267028183, 0.012562861976226226, 5.0e-324]
new_d = NegativeBinomial{Float64}(r=2.1996727803865155, p=0.5)
gcm.θ = [0.6698378267028183, 9.99999999, 9.99999999]
This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        4

Total number of variables............................:        3
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with onl

LoadError: InterruptException:

In [57]:
@show β_true
@show gcm.β
@show gcm.Σ
@show gcm.∇β
@show gcm.data[1].d
@show GLMCopula.loglikelihood!(gcm, true, true);

β_true = 0.5389965007326872
gcm.β = [0.5038358480381085]
gcm.Σ = [0.5459747582737707, 0.0]
gcm.∇β = [-1.1747502495218498e-6]
(gcm.data[1]).d = NegativeBinomial{Float64}(r=2.457944258588164, p=0.5)
GLMCopula.loglikelihood!(gcm, true, true) = -91803.25750220718


In [14]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2");

estimated β = -0.8574308284816735; true β = -0.8472978603872034
estimated variance component 1 = 0.982527024613311; true variance component 1 = 0.9
estimated variance component 2 = 0.12942862508284544; true variance component 2 = 0.1


In [9]:
# still needs some work (I hypothesize from GLM loglikeobs function)
using BenchmarkTools

@benchmark loglikelihood!($gcm, true, true)

BenchmarkTools.Trial: 
  memory estimate:  22.89 MiB
  allocs estimate:  250000
  --------------
  minimum time:     31.666 ms (0.00% GC)
  median time:      34.955 ms (0.00% GC)
  mean time:        38.332 ms (6.67% GC)
  maximum time:     102.631 ms (60.42% GC)
  --------------
  samples:          131
  evals/sample:     1